#### Функция super()

In [2]:
from collections import OrderedDict

class LastUpdatedOrderedDict(OrderedDict):
    """Элементы хранятся в порядке, определяемом последним обновлением"""
    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.move_to_end(key)

In [3]:
class NotRecommended(OrderedDict):
    """Так поступать не рекомендуется!"""
    def __setitem__(self, key, value):
        OrderedDict.__setitem__(self, key, value)
        self.move_to_end(key)

In [4]:
class LastUpdatedOrderedDict(OrderedDict):
    """Этот код работает в Python 2 и в Python 3"""
    def __setitem__(self, key, value):
        super(LastUpdatedOrderedDict, self).__setitem__(key, value)
        self.move_to_end(key)

#### Пример 14.1. Наш метод __setitem__ игнорируется методами __init__ и __update__ встроенного типа dict

In [5]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict(one=1)
dd

{'one': 1}

In [6]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [7]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

#### Пример 14.2. Метод __getitem__ из класса AnswerDict игнорируется методом dict.update

In [10]:
class AnswerDict:
    def __init__(self, *args, **kwargs):
        pass
        
    def __getitem__(self, key):
        return 42

ad = AnswerDict(a='foo')
ad['a']

42

In [11]:
d = {}
d.update(ad)
d['a']

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [12]:
d

{}

#### Пример 14.3. Классы DoppelDict2 и AnswerDict2 работают, как и ожидалось, потому что расширяют UserDict, а не dict

In [14]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [15]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [16]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [17]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict2(a='foo')
ad['a']

42

In [18]:
d = {}
d.update(ad)
d['a']

42

In [19]:
d

{'a': 42}

#### Пример 14.4. diamond.py: классы Leaf, A, B, Root образуют граф, показанный на рис. 14.1

In [20]:
class Root:  # <1>
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root):  # <2>
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root):  # <3>
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B):  # <4>
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

#### Пример 14.5. Тесты вызова методов ping и pong объекта Leaf

In [21]:
leaf1 = Leaf()
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [22]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


In [23]:
Leaf.__mro__ # doctest:+NORMALIZE_WHITESPACE

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

#### Пример 14.6. diamond2.py: классы, демонстрирующие динамическую природу super()

In [24]:
class U():  # <2>
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()  # <3>

class LeafUA(U, A):  # <4>
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()
# end::DIAMOND_CLASSES[]

class LeafAU(A, U):
    def ping(self):
        print(f'{self}.ping() in LeafAU')
        super().ping()

In [25]:
u = U()
u.ping()

<__main__.U object at 0x7f5888110700>.ping() in U


AttributeError: 'super' object has no attribute 'ping'

In [26]:
leaf2 = LeafUA()
leaf2.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


In [27]:
LeafUA.__mro__ # doctest:+NORMALIZE_WHITESPACE

(__main__.LeafUA, __main__.U, __main__.A, __main__.Root, object)

#### Пример 14.7. MRO класса tkinter.Text

In [28]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))


import tkinter
print_mro(tkinter.Text)

ModuleNotFoundError: No module named 'tkinter'

#### Пример 14.8. uppermixin.py: UpperCaseMixin поддерживает отображения, не зависящие от регистра

In [29]:
import collections

def _upper(key):  # <1>
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:  # <2>
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

#### Пример 14.9. uppermixin.py: два класса, использующих UpperCaseMixin

In [30]:
class UpperDict(UpperCaseMixin, collections.UserDict):  # <1>
    pass

class UpperCounter(UpperCaseMixin, collections.Counter):  # <2>
    """Specialized 'Counter' that uppercases string keys"""  # <3>

In [31]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])
list(d.keys())

['A', 2]

In [32]:
d['b'] = 'letter B'
'b' in d

True

In [33]:
d['a'], d.get('B')

('letter A', 'letter B')

In [34]:
list(d.keys())

['A', 2, 'B']

In [35]:
c = UpperCounter('BaNanA')
c.most_common()

[('A', 3), ('N', 2), ('B', 1)]

#### Пример 14.10. Часть Lib/socketserver.py в Python 3.10

In [ ]:
class ThreadingMixIn:
    """Mix-in class to handle each request in a new thread."""

    # Decides how threads will act upon termination of the
    # main process
    daemon_threads = False
    # If true, server_close() waits until all non-daemonic threads terminate.
    block_on_close = True
    # Threads object
    # used by server_close() to wait for all threads completion.
    _threads = _NoThreads()

    def process_request_thread(self, request, client_address):
        """Same as in BaseServer but as a thread.

        In addition, exception handling is done here.

        """
        try:
            self.finish_request(request, client_address)
        except Exception:
            self.handle_error(request, client_address)
        finally:
            self.shutdown_request(request)

    def process_request(self, request, client_address):
        """Start a new thread to process the request."""
        if self.block_on_close:
            vars(self).setdefault('_threads', _Threads())
        t = threading.Thread(target = self.process_request_thread,
                             args = (request, client_address))
        t.daemon = self.daemon_threads
        self._threads.append(t)
        t.start()

    def server_close(self):
        super().server_close()
        self._threads.join()